# Convert Open Images into Pascal VOC

## Download only selected classes from Open Images

Clone the `OIDv4_ToolKit` repo

```
!git clone https://github.com/EscVM/OIDv4_ToolKit
```

cd into `OIDv4_ToolKit` and run `python3 main.py downloader --classes {class} --type_csv all`

```
!cd OIDv4_ToolKit && python3 main.py downloader --classes Coin --type_csv all -y
```

We need to download extra files and place them in `~/OIDv4_ToolKit/OID/csv_folder`

```
!wget https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv
!wget https://storage.googleapis.com/openimages/2018_04/validation/validation-images-with-rotation.csv
!wget https://storage.googleapis.com/openimages/2018_04/test/test-images-with-rotation.csv
```

## Convert OID to XML

**Location:** 
OIDv4_ToolKit parent directory

**Usage:**
Start from OIDv4_ToolKit root directory.

The script will create directories called To_PASCAL_XML (similar to the Label directories) in the Dataset Subdirectories.
These directories contain the XML files.
You need to download the Images and generate ToolKit-Style-Labels via the OIDv4_ToolKit before using this script.

Run
```
!cd OIDv4_ToolKit && python oid_to_pascal_voc_xml.py
```

In [ ]:
%%writefile /home/ec2-user/SageMaker/object-detection/OIDv4_ToolKit/oid_to_pascal_voc_xml.py

"""
Location: OIDv4_ToolKit parent directory

Usage: 
    Start from OIDv4_ToolKit root directory.

    The script will create directories called To_PASCAL_XML (similar to the Label directories) in the Dataset Subdirectories.
    These directories contain the XML files.
    You need to download the Images and generate ToolKit-Style-Labels via the OIDv4_ToolKit before using this script.
    """

import os
from tqdm import tqdm
from sys import exit
import argparse
import cv2
from textwrap import dedent
from lxml import etree


XML_DIR = 'To_PASCAL_XML'


os.chdir(os.path.join("OID", "Dataset"))
DIRS = os.listdir(os.getcwd())

for DIR in DIRS:
    if os.path.isdir(DIR):
        os.chdir(DIR)

        print("Currently in Subdirectory:", DIR)

        CLASS_DIRS = os.listdir(os.getcwd())
        
        for CLASS_DIR in CLASS_DIRS:
            if os.path.isdir(CLASS_DIR):
                os.chdir(CLASS_DIR)

                print("\n" + "Creating PASCAL VOC XML Files for Class:", CLASS_DIR)
                # Create Directory for annotations if it does not exist yet
                if not os.path.exists(XML_DIR):
                    os.makedirs(XML_DIR)

                #Read Labels from OIDv4 ToolKit
                os.chdir("Label")

                #Create PASCAL XML
                for filename in tqdm(os.listdir(os.getcwd())):
                    if filename.endswith(".txt"):
                        filename_str = str.split(filename, ".")[0]


                        annotation = etree.Element("annotation")
                        
                        os.chdir("..")
                        folder = etree.Element("folder")
                        folder.text = os.path.basename(os.getcwd())
                        annotation.append(folder)

                        filename_xml = etree.Element("filename")
                        filename_xml.text = filename_str + ".jpg"
                        annotation.append(filename_xml)

                        path = etree.Element("path")
                        path.text = os.path.join(os.path.dirname(os.path.abspath(filename)), filename_str + ".jpg")
                        annotation.append(path)

                        source = etree.Element("source")
                        annotation.append(source)

                        database = etree.Element("database")
                        database.text = "Unknown"
                        source.append(database)

                        size = etree.Element("size")
                        annotation.append(size)

                        width = etree.Element("width")
                        height = etree.Element("height")
                        depth = etree.Element("depth")

                        img = cv2.imread(filename_xml.text)

                        width.text = str(img.shape[1])
                        height.text = str(img.shape[0])
                        depth.text = str(img.shape[2])

                        size.append(width)
                        size.append(height)
                        size.append(depth)

                        segmented = etree.Element("segmented")
                        segmented.text = "0"
                        annotation.append(segmented)

                        os.chdir("Label")
                        label_original = open(filename, 'r')

                        # Labels from OIDv4 Toolkit: name_of_class X_min Y_min X_max Y_max
                        for line in label_original:
                            line = line.strip()
                            l = line.split(' ')
                            class_name = l[0]
                            xmin_l = str(int(float(l[1])))
                            ymin_l = str(int(float(l[2])))
                            xmax_l = str(int(float(l[3])))
                            ymax_l = str(int(float(l[4])))
                            
                            obj = etree.Element("object")
                            annotation.append(obj)

                            name = etree.Element("name")
                            name.text = class_name
                            obj.append(name)

                            pose = etree.Element("pose")
                            pose.text = "Unspecified"
                            obj.append(pose)

                            truncated = etree.Element("truncated")
                            truncated.text = "0"
                            obj.append(truncated)

                            difficult = etree.Element("difficult")
                            difficult.text = "0"
                            obj.append(difficult)

                            bndbox = etree.Element("bndbox")
                            obj.append(bndbox)

                            xmin = etree.Element("xmin")
                            xmin.text = xmin_l
                            bndbox.append(xmin)

                            ymin = etree.Element("ymin")
                            ymin.text = ymin_l
                            bndbox.append(ymin)

                            xmax = etree.Element("xmax")
                            xmax.text = xmax_l
                            bndbox.append(xmax)

                            ymax = etree.Element("ymax")
                            ymax.text = ymax_l
                            bndbox.append(ymax)

                        os.chdir("..")

                        os.chdir(XML_DIR)

                        # write xml to file
                        s = etree.tostring(annotation, pretty_print=True)
                        with open(filename_str + ".xml", 'wb') as f:
                            f.write(s)
                            f.close()

                        os.chdir("..")
                        os.chdir("Label")

                os.chdir("..")
                os.chdir("..")   
                   
        os.chdir("..")
                   

## Create VOC_ROOT folder structure and populate

### Import Libraries

In [57]:
import os
import shutil

from PIL import Image
from pathlib import Path
from glob import glob

### Create .txt and copy files over

In [58]:
path_to_OID = Path.cwd()/'object-detection/OIDv4_ToolKit/OID/'

In [62]:
image_sets = path_to_OID/'VOC_ROOT/VOC2012/ImageSets/Main'
jpeg_images = path_to_OID/'VOC_ROOT/VOC2012/JPEGImages'
annotations = path_to_OID/'VOC_ROOT/VOC2012/Annotations'

image_sets.mkdir(parents=True, exist_ok=True)
jpeg_images.mkdir(parents=True, exist_ok=True)
annotations.mkdir(parents=True, exist_ok=True)

for split in ['train','test','val']:
    list_images, list_xml = [],[]
    list_images = [x for x in (path_to_OID/f'Dataset/{split}/Coin').glob('*.jpg')]
    list_xml = [x for x in (path_to_OID/f'Dataset/{split}/Coin/To_PASCAL_XML').glob('*.xml')]
    
    with open(image_sets/f'{split}.txt', 'w') as f:
        for item in list_images:
            f.write("%s\n" % item.stem)
    
    for item in list_images:
        shutil.copyfile(item, jpeg_images/f'{item.name}')  
        
    for item in list_xml:
        shutil.copyfile(item, annotations/f'{item.name}')
        
with open(image_sets/'trainval.txt', 'wb') as wfd:
    for f in [image_sets/'train.txt',image_sets/'val.txt']:
        with open(f,'rb') as fd:
            shutil.copyfileobj(fd, wfd)

## Train SSD

The training with PASCAL VOC datasets need the class names to be in lower case in the XML so we are going to replace them in all files inside `VOC_ROOT/VOC2012/Annotations` with:
```
!find ./ -type f -exec sed -i 's/Coin/coin/' {} \;
```

Git clone the following SSD implementation in PyTorch:
```
git clone https://github.com/lufficc/SSD
```

We need to edit the configuration file (.yaml): ie. `vgg_ssd300_voc0712.yaml` or `efficient_net_b3_ssd300_voc0712.yaml` and change `NUM_CLASSES:` from 21 to 2.

We now edit `./SSD/ssd/data/datasets/voc.py` and change:

```
class_names = ('__background__',
               'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor')
```

to

```
class_names = ('__background__','coin')
```

We can start training with:

```
# for example, train SSD300:
python train.py --config-file configs/vgg_ssd300_voc0712.yaml --use_tensorboard 0
```